In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
import operator
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
os.chdir("/Users/Yanean/Desktop/ML/data")

In [ ]:
def load_features(filepath):
    feature = np.array(pd.read_csv(filepath, header = None))
    feature_dict = {}
    for i in range(len(feature)):
        id = int(feature[i][0].split("/")[1].split(".")[0])
        feature_dict[id] = np.array(feature[i][1:], dtype=float)
    res = []
    for i in range(len(feature_dict)):
        res.append(feature_dict[i])
    return np.array(res)

In [ ]:
train_featfc = load_features("./features_train/features_resnet1000_train.csv")
test_featfc = load_features("./features_test/features_resnet1000_test.csv")
train_featpool=load_features("./features_train/features_resnet1000intermediate_train.csv")
test_featpool=load_features("./features_test/features_resnet1000intermediate_test.csv")

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
total_tag_dict = list()
def get_tag_dict(path):
    # open all description trianing file(5 sentences txt)
    for filename in glob.glob(os.path.join(path, "*.txt")):
        with open(filename, "r") as content_file:
            content = content_file.read()
            sentence = content.split("\n")
            for k in sentence:
                sentence2 = k.split(":")
                for str in sentence2:
                    if str not in total_tag_dict and str != "":
                        total_tag_dict.append(str)
get_tag_dict("./tags_train/")
print (len(total_tag_dict))
print (total_tag_dict)

In [ ]:
tag_dict = {} # HashMap (word, id), word stands for high frequency word while id is assigned to it
index = 0
for w in total_tag_dict:
    tag_dict[w] = index
    index += 1 
print (len(tag_dict))

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
total_word_dict = {}
def get_word_dict(path):
    # open all description trianing file(5 sentences txt)
    for filename in glob.glob(os.path.join(path, "*.txt")):
        with open(filename, "r") as content_file:
            content = content_file.read()
            # Lemmatizing & Remove Stop Words
            sentence = filter(None, re.split(r'[^a-zA-Z\']', content.lower()))
            stop = set(stopwords.words('english'))
            #print stop
            lmtzr = WordNetLemmatizer()
            parsed = [lmtzr.lemmatize(word) for word in sentence if word not in stop]
            #print parsed
            for word in parsed:
                #w = stemmer.stem(word)
                if word in total_word_dict:
                    total_word_dict[word] += 1
                else:
                    total_word_dict[word] = 1
# get the word dictionary for training description
# HashMap: (word, frequency)
get_word_dict("./descriptions_train/")

print (total_word_dict)

In [ ]:
word_dict = {} # HashMap (word, id), word stands for high frequency word while id is assigned to it
index = 0
# for all words in dictionary, if frequency > threshold, identify it as a high frequency word 
# assign an index to it, and increase the index
for w in total_word_dict:
    if total_word_dict[w] > 10:
        word_dict[w] = index
        index += 1 
print (word_dict)
print (len(word_dict))

In [ ]:
total_dict = {}
index = 0
for w in word_dict:
    if w not in total_dict:
        total_dict[w] = index
        index = index + 1
for w in tag_dict:
    if w not in total_dict:
        total_dict[w] = index
        index = index + 1
        
# print len(total_dict)
# print (total_dict)

def process_des(path, thresh):
    #HashMap (filename, array)
    # filename example could be 1.txt etc. 
    # array should be a 1644 length, telling us for this specific file (ex. 1.txt)
    # the frequency of all high frequency word (for example, the word yellow appears 12 times in total)
    # but in the current file, it appears 3 times
    des_vec = {}
    for filename in glob.glob(os.path.join(path, "*.txt")):
        with open(filename, "r") as content_file:
            content = content_file.read()
            # Lemmatizing & Remove Stop Words
            cur = [0] * len(total_dict)
            sentence = filter(None, re.split(r'[^a-zA-Z\']', content.lower()))
            stop = set(stopwords.words('english'))
            #print stop
            lmtzr = WordNetLemmatizer()
            parsed = [lmtzr.lemmatize(word) for word in sentence if word not in stop]
            #print parsed
            for word in parsed:
                #w = stemmer.stem(word)
                if word in total_dict:
                    cur[total_dict[word]] = 1
            des_vec[filename.split('/')[-1]] = cur
    return des_vec # ex. {"0.txt": [1,0,0,2,0,0,0,0,.....]; "1.txt" : [2,1,1,1,0,0,1,....]}

thresh = 10
train_des = process_des("./descriptions_train/", thresh)
print (train_des["0.txt"])
print (len(train_des["0.txt"]))
test_des = process_des("./descriptions_test/", thresh)
#print test_des["0.txt"]

In [ ]:
def process_des(path):
    des_vec = {}
    for filename in glob.glob(os.path.join(path, "*.txt")):
        with open(filename, "r") as content_file:
            content = content_file.read()
            sentence = content.split("\n")
            cur = [0] * len(total_tag_dict)
            for k in sentence:
                sentence2 = k.split(":")
                for str in sentence2:
                    #print str
                    if str in total_tag_dict:
                        cur[tag_dict[str]] = 1
            des_vec[filename.split('/')[-1]] = cur
    return des_vec 

train_tag_des = process_des("./tags_train/")
print (type(train_tag_des["0.txt"]))
test_tag_des = process_des("./tags_test/")
#print test_des["0.txt"]

In [ ]:
Y = [ ]
for k in train_tag_des:
    Y.append(train_tag_des[k][:-3])
# print(Y)
len(Y)
Y = np.array(Y, dtype = np.float32)
#      np.concatenate((Y, train_tag_des[k]), axis=0)
    #Y.append(train_tag_des[k])
#     filename = int(k.split(".")[0])
    #print len(train_featfc_map[filename].reshape(-1, 1))
#     Y = np.concatenate((Y, train_featfc_map[filename]), axis=0)
#     Y = np.concatenate((Y, train_featpool_map[filename]), axis=0)
    #Y.append(train_featpool_map[filename])
#     print k
#     print i
#     i = i +1
# Y = np.array(Y, dtype = np.float32)

Y= np.concatenate((Y, train_featfc), axis=1)
Y = np.concatenate((Y, train_featpool), axis=1)
Y = np.array(Y, dtype = np.float32)
# print (Y.shape)

In [ ]:
# first part for SVM
X = []
for k in train_des:
    #train_des[k].append(k)
    X.append(train_des[k])
X = np.array(X, dtype = np.float32)
print (X.shape)

In [12]:
from sklearn.neural_network import MLPRegressor
# train_x = total_bow[0:10000] # 这里是training的description
# train_y = vis # 这里是train的resNet
# 在这里加入模型
reg = MLPRegressor(hidden_layer_sizes=(1000,1000,1000),max_iter = 500,alpha=0.0001,
                   solver='sgd',verbose=10,random_state=21,tol = 0.0000001)
# reg = MLPRegressor(solver='sgd', activation='logistic', shuffle=True)
reg.fit(X, Y)
p = reg.predict(test_des["0.txt"])
print (p)

Iteration 1, loss = 0.76393553
Iteration 2, loss = 0.71543404
Iteration 3, loss = 0.71175271
Iteration 4, loss = 0.70606606
Iteration 5, loss = 0.69744292
Iteration 6, loss = 0.68407222
Iteration 7, loss = 0.66627243
Iteration 8, loss = 0.65002539
Iteration 9, loss = 0.63140882
Iteration 10, loss = 0.59320141
Iteration 11, loss = 0.55206068
Iteration 12, loss = 0.51918393
Iteration 13, loss = 0.48775329
Iteration 14, loss = 0.45677375
Iteration 15, loss = 0.43511087
Iteration 16, loss = 0.41895702
Iteration 17, loss = 0.40445130
Iteration 18, loss = 0.39101383
Iteration 19, loss = 0.37955579
Iteration 20, loss = 0.36868184
[[ 0.22834304  0.04068868  0.14662281 ...,  0.27115732  0.56784736
   0.36727272]]


/Applications/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:565: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/Applications/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [13]:
#print p.shape
X_test = []
for k in test_des:
    #train_des[k].append(k)
    X_test.append(test_des[k])
X_test = np.array(X_test, dtype = np.float32)
print (X_test.shape)

Y_test = []
for k in test_tag_des:
    #train_tag_des[k].append(k)
    #print k
    # k的value很重要 你看到k就知道这是哪一个txt，然后你根据k的value去hashmap里面找相关test_featpc和test_featpool的vector
    # 这个hashmap 你在一开始读入文件的时候就可以build一下  我把第0行给删了 其实第0行还是有些用处的 可以用来build hashmap
    Y_test.append(test_tag_des[k][:-3])
Y_test = np.array(Y_test, dtype = np.float32)
print (Y_test.shape)
print(test_featfc.shape)
print(test_featpool.shape)
Y_test = np.concatenate((Y_test, test_featfc), axis=1)
Y_test = np.concatenate((Y_test, test_featpool), axis=1)
print (Y_test.shape)

(2000, 1822)
(10000, 88)
(2000, 1000)
(2000, 2048)


ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [ ]:
pred = []
for k in range(2000):
    fileName = str(k)+'.txt'
    p = reg.predict(np.array(test_des[fileName]).reshape(1,-1))
    pred.append(p[0])

In [ ]:
pred = []
for k in range(2000):
    fileName = str(k)+'.txt'
    p = reg.predict(np.array(test_des[fileName]).reshape(1,-1))
    pred.append(p[0])

In [ ]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=20, algorithm='ball_tree').fit(Y_test)
print(pred)
result=nbrs.kneighbors(np.array(pred))



In [ ]:
import time,csv
def get_timestamp():
    millis = int(round(time.time() * 1000))
    return millis

def create_submission_csv(predictions):
    filename = 'submission' + '_' + str(get_timestamp()) + '.csv'
    with open(filename, 'w') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['Descritpion_ID', 'Top_20_Image_IDs'])
        for i in range (0, len(predictions)):
            description_id = str(i) + ".txt"
            top20_image_ids = " ".join(str(index) + '.jpg' for index in predictions[i])
            writer.writerow([description_id, top20_image_ids])
create_submission_csv(result[1])            